In [5]:
import tensorflow as tf
import layer
import loss
import tensorflow_datasets as tfds


NotFoundError: dlopen(/Users/zarizky/conda-envs/tensorflow/lib/python3.9/site-packages/tensorflow-plugins/libmetal_plugin.dylib, 0x0006): symbol not found in flat namespace '__ZN3tsl8internal10LogMessage16VmoduleActivatedEPKci'

In [ ]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Input((28, 28, 3)))
model.add(tf.keras.layers.Conv2D(256, 9))
model.add(tf.keras.layers.Activation('relu'))
model.add(layer.PrimaryCapsule(32, 8, 9, 2, activation='dr'))
model.add(layer.DRCapsule(10, 16, activation='dr'))
model.compile(loss=loss.MarginLoss(), metrics='categorical_accuracy', optimizer='adam')
model.summary()

In [ ]:
(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

In [ ]:
def normalize_img(image, label):
    """Normalizes images: `uint8` -> `float32`."""
    return tf.cast(image, tf.float32) / 255., label


ds_train = ds_train.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(128)
ds_train = ds_train.prefetch(tf.data.AUTOTUNE)

ds_test = ds_test.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_test = ds_test.batch(128)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.AUTOTUNE)

In [ ]:
model.fit(
    ds_train,
    epochs=6,
    validation_data=ds_test,
)